In [1]:
import pandas as pd
import requests
import json

class ScriptData:
    def __init__(self):
        self.data = {}
    
    def fetch_intraday_data(self, script):
        # API Key value from link
        api_key = 'GF0IU3N4VNWYWAV5'
        
        # construct the API URL
        base_url = "https://www.alphavantage.co/query"
        function = "TIME_SERIES_INTRADAY"
        symbol = script
        interval = "60min"
        outputsize = "compact"
        api_url = f"{base_url}?function={function}&symbol={symbol}&interval={interval}&outputsize={outputsize}&apikey={api_key}"
        
        # send the request and parse the response
        response = requests.get(api_url)
        if response.status_code != 200:
            raise Exception("Failed to fetch data. Check the API key and script name.")
        data = response.json()
        self.data[script] = data
    
    def convert_intraday_data(self, script):
        # get the data for the given script
        data = self.data[script]
        
        # extract the time series data
        time_series_data = data["Time Series (60min)"]
        
        # create a list of tuples containing the timestamp and data for each time point
        data_list = [(timestamp, data) for timestamp, data in time_series_data.items()]
        
        # sort the data by timestamp
        data_list.sort(key=lambda x: x[0])
        
        # extract the data into separate lists
        timestamps, data = zip(*data_list)
        opens, highs, lows, closes, volumes = [], [], [], [], []
        for d in data:
            opens.append(float(d["1. open"]))
            highs.append(float(d["2. high"]))
            lows.append(float(d["3. low"]))
            closes.append(float(d["4. close"]))
            volumes.append(int(d["5. volume"]))
        
        # create a pandas DataFrame from the lists
        df = pd.DataFrame({"timestamp": timestamps, "open": opens, "high": highs, "low": lows, "close": closes, "volume": volumes})
        
        # set the timestamp column as the index of the DataFrame
        df.set_index("timestamp", inplace=True)
        
        # convert the timestamp strings to pandas Timestamps
        df.index = pd.to_datetime(df.index)
        
        # store the DataFrame in the data dictionary
        self.data[script] = df
    
    def __getitem__(self, key):
        return self.data[key]
    
    def __setitem__(self, key, value):
        self.data[key] = value
    
    def __contains__(self, key):
        return key in self.data


In [2]:
# create an instance of the ScriptData class
script_data = ScriptData()

# fetch intraday data for the 'GOOGL' script using your Alpha Vantage API key
script_data.fetch_intraday_data('GOOGL')

# convert the fetched data into a pandas DataFrame
script_data.convert_intraday_data('GOOGL')

# access the DataFrame for the 'GOOGL' script
df = script_data['GOOGL']




In [3]:
df

,open,high,low,close,volume
timestamp,,,,,
2023-01-05 17:00:00,86.20,86.4541,86.1000,86.2200,882007
2023-01-05 18:00:00,86.24,86.2500,86.1500,86.1800,13265
2023-01-05 19:00:00,86.24,86.3000,86.2100,86.2999,10652
2023-01-05 20:00:00,86.25,86.3600,86.2100,86.3600,10801
2023-01-06 05:00:00,86.45,86.4500,85.9900,86.1400,10025
...,...,...,...,...,...
2023-01-13 16:00:00,91.61,92.1900,91.4941,92.0800,5239221
2023-01-13 17:00:00,92.12,92.1200,91.7700,91.9200,2874727
2023-01-13 18:00:00,91.86,92.1200,91.8100,91.8500,40290


In [4]:

script_data.fetch_intraday_data('AAPL')

script_data.convert_intraday_data('AAPL')

df = script_data['AAPL']





In [5]:
df

,open,high,low,close,volume
timestamp,,,,,
2023-01-05 17:00:00,125.02,125.34,124.99,125.16,2218681
2023-01-05 18:00:00,125.15,125.15,125.05,125.10,75097
2023-01-05 19:00:00,125.13,125.23,124.80,125.17,88894
2023-01-05 20:00:00,125.17,125.25,125.12,125.25,46537
2023-01-06 05:00:00,125.52,125.52,124.64,124.96,57740
...,...,...,...,...,...
2023-01-13 16:00:00,134.14,134.92,134.07,134.70,9715584
2023-01-13 17:00:00,134.70,134.78,134.42,134.52,1705474
2023-01-13 18:00:00,134.59,134.63,134.52,134.59,23742


In [6]:
'GOOGL' in script_data

True

In [7]:
'AAPL' in script_data

True

In [8]:
'NVDA' in script_data

False

In [9]:
def indicator1(df, timeperiod):
    # create a new column in the DataFrame with the moving average of the close price
    df["indicator"] = df["close"].rolling(timeperiod).mean()
    
    # create a new DataFrame with the timestamp and indicator columns
    result = pd.DataFrame({"timestamp": df.index, "indicator": df["indicator"]})

    return result


In [10]:
# fetch and convert intraday data for the 'GOOGL' script
script_data = ScriptData()
script_data.fetch_intraday_data('GOOGL')
script_data.convert_intraday_data('GOOGL')

# compute the indicator1 for the 'GOOGL' script with a timeperiod of 5
indicator1_df1 = indicator1(script_data['GOOGL'], timeperiod = 5)


In [11]:
indicator1_df1

,timestamp,indicator
timestamp,,
2023-01-05 17:00:00,2023-01-05 17:00:00,NaN
2023-01-05 18:00:00,2023-01-05 18:00:00,NaN
2023-01-05 19:00:00,2023-01-05 19:00:00,NaN
2023-01-05 20:00:00,2023-01-05 20:00:00,NaN
2023-01-06 05:00:00,2023-01-06 05:00:00,86.23998
...,...,...
2023-01-13 16:00:00,2023-01-13 16:00:00,91.44032
2023-01-13 17:00:00,2023-01-13 17:00:00,91.67632
2023-01-13 18:00:00,2023-01-13 18:00:00,91.80700


In [12]:
script_data = ScriptData()
script_data.fetch_intraday_data('AAPL')
script_data.convert_intraday_data('AAPL')

indicator1_df2 = indicator1(script_data['AAPL'], timeperiod = 5)

In [13]:
indicator1_df2

,timestamp,indicator
timestamp,,
2023-01-05 17:00:00,2023-01-05 17:00:00,NaN
2023-01-05 18:00:00,2023-01-05 18:00:00,NaN
2023-01-05 19:00:00,2023-01-05 19:00:00,NaN
2023-01-05 20:00:00,2023-01-05 20:00:00,NaN
2023-01-06 05:00:00,2023-01-06 05:00:00,125.12800
...,...,...
2023-01-13 16:00:00,2023-01-13 16:00:00,133.94390
2023-01-13 17:00:00,2023-01-13 17:00:00,134.20528
2023-01-13 18:00:00,2023-01-13 18:00:00,134.41548


In [14]:

class Strategy:
    def __init__(self, script):
        self.script_data = ScriptData()
    
    def fetch_intraday_data(self, script):
        self.script_data.fetch_intraday_data(script)
        self.script_data.convert_intraday_data(script)
        self.df = self.script_data[script]
    
    def compute_indicator_data(self):
        self.indicator_df = indicator1(self.df, 5)
    
    def get_signals_data(self):
        # create a new column in the DataFrame with the moving average of the close price
        self.df["indicator"] = self.indicator_df["indicator"]
        
        # create a new column in the DataFrame with the signals
        self.df["signal"] = "NO_SIGNAL"
        self.df.loc[self.df["indicator"] > self.df["close"], "signal"] = "BUY"
        self.df.loc[self.df["indicator"] < self.df["close"], "signal"] = "SELL"
        
        # create a new DataFrame with the timestamp and signal columns
        self.initial_signals = pd.DataFrame({"timestamp": self.df.index, "signal": self.df["signal"]})
        
        # filter the signals DataFrame to include only the rows with a 'BUY' or 'SELL' signal
        self.signals = strategy.initial_signals[strategy.initial_signals["signal"].isin(["BUY", "SELL"])]
        
        # track the previous signal
        prev_signal = self.signals.iloc[0]["signal"]
        
        # create a new DataFrame to store the timestamps and signals
        self.signals_df = pd.DataFrame(columns=["timestamp", "signal"])
        
        # iterate through the signals DataFrame and add the timestamp and signal to the signals_df DataFrame if the signal is different from the previous signal
        for i, row in self.signals.iterrows():
            if row["signal"] != prev_signal:
                # update the signals_df DataFrame by concatenating the current row
                self.signals_df = pd.concat([self.signals_df, pd.DataFrame({"timestamp": [row["timestamp"]], "signal": [row["signal"]]})])
            prev_signal = row["signal"]
            
        

        return self.signals_df

        


In [15]:
# create an instance of the Strategy class
strategy = Strategy('NVDA')

# fetch intraday data for the script 'NVDA'
strategy.fetch_intraday_data('NVDA')

# compute the indicator data
strategy.compute_indicator_data()

# get the signals data

signals_df = strategy.get_signals_data()

signals_df = signals_df.reset_index(drop=True)

In [16]:
signals_df

,timestamp,signal
0,2023-01-06 09:00:00,SELL
1,2023-01-06 10:00:00,BUY
2,2023-01-06 11:00:00,SELL
3,2023-01-06 18:00:00,BUY
4,2023-01-06 20:00:00,SELL
5,2023-01-09 14:00:00,BUY
6,2023-01-10 05:00:00,SELL
7,2023-01-10 07:00:00,BUY
8,2023-01-10 10:00:00,SELL
9,2023-01-11 08:00:00,BUY
